Oh My Zsh - Q/A Agent 

In [7]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from memory import get_memory

ohmyzshwiki_store = get_memory().vectordb

print([x.metadata for x in ohmyzshwiki_store.as_retriever().get_relevant_documents("change shell color")])

[{'file_path': 'External-themes.md', 'file_name': 'External-themes.md', 'file_type': '.md'}, {'file_path': 'External-themes.md', 'file_name': 'External-themes.md', 'file_type': '.md'}, {'file_path': 'External-themes.md', 'file_name': 'External-themes.md', 'file_type': '.md'}]


Initalize Agent

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="oh my zsh wiki",
    description="oh my zsh wiki vectorstore from github repo",
    vectorstore=ohmyzshwiki_store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

vector_agent = create_vectorstore_agent(
    llm=ChatOpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

Example Runs 

In [32]:
vector_agent.run("What is oh my zsh?")



> Entering new AgentExecutor chain...
I should use the oh my zsh wiki to answer this question.
Action: oh my zsh wiki
Action Input: "What is oh my zsh?"
Observation:  Oh My Zsh is a command line framework for managing your zsh configuration. It includes features such as themes, plugins, and autocompletion that make it easier to use the terminal.
Thought:I have the answer, but the user did not ask for sources, so I can just provide the answer.
Final Answer: Oh My Zsh is a command line framework for managing your zsh configuration. It includes features such as themes, plugins, and autocompletion that make it easier to use the terminal.

> Finished chain.


'Oh My Zsh is a command line framework for managing your zsh configuration. It includes features such as themes, plugins, and autocompletion that make it easier to use the terminal.'

In [36]:
vector_agent.run("hi")



> Entering new AgentExecutor chain...
This question is not relevant to any of the tools provided.
Action: N/A
Final Answer: I don't know.

> Finished chain.


"I don't know."

In [42]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationSummaryBufferMemory

tools = [
    Tool(
        name = "Intermediate Answer",
        func=get_memory().agent_tool(),
        description="This is your primary tool. You always start here. This is the Oh My Zsh (ohmyzsh) wiki. It may not have all the info."

    ),
 ]
memory = ConversationSummaryBufferMemory(memory_key="chat_history", 
                                         llm=OpenAI(temperature=0),
                                         return_messages=True)

convo_agent = initialize_agent(tools=tools, 
                              llm=OpenAI(temperature=0),
                              agent=AgentType.SELF_ASK_WITH_SEARCH,
                              verbose=True)

In [43]:
convo_agent.run(input="What is oh my zsh?")



> Entering new AgentExecutor chain...
 No.
So the final answer is: Oh My Zsh is an open source, community-driven framework for managing your zsh configuration.

> Finished chain.


'Oh My Zsh is an open source, community-driven framework for managing your zsh configuration.'

In [44]:
convo_agent.run(input="hi, whats up?")



> Entering new AgentExecutor chain...
 No.
So the final answer is: Hi! How can I help you?

> Finished chain.


'Hi! How can I help you?'